In [4]:
# Customer Clustering Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from kneed import KneeLocator

# Veriyi yükle (feature engineering'den sonra)
customer_data = pd.read_csv('../results/customer_features.csv')

print("🎯 MÜŞTERİ KÜMELEME ANALİZİ")
print("="*50)

# Clustering için özellikler seç
clustering_features = ['Recency', 'Frequency', 'Monetary',
                      'Total_Spent', 'Avg_Order_Value', 'Unique_Products',
                      'Customer_Lifespan_Days', 'Purchase_Frequency']

# Eksik değerleri doldur
X = customer_data[clustering_features].fillna(0)

# Özellikleri standartlaştır
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"📊 {len(X)} müşteri, {len(clustering_features)} özellik ile kümeleme yapılıyor")

# 1. Optimal küme sayısını bul
def find_optimal_clusters(data, max_k=15):
    """Elbow method ve Silhouette score ile optimal küme sayısı"""

    sse = []
    silhouette_scores = []
    k_range = range(2, max_k + 1)

    print("🔍 Optimal küme sayısı aranıyor...")

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(data)

        sse.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(data, kmeans.labels_))

        print(f"K={k}: SSE={kmeans.inertia_:.0f}, Silhouette={silhouette_score(data, kmeans.labels_):.3f}")

    # Elbow point bul
    kl = KneeLocator(k_range, sse, curve="convex", direction="decreasing")
    elbow_k = kl.elbow if kl.elbow else 4

    # En yüksek silhouette score
    best_silhouette_k = k_range[np.argmax(silhouette_scores)]

    print(f"\n📈 Elbow method önerisi: {elbow_k} küme")
    print(f"📈 En iyi Silhouette score: {best_silhouette_k} küme (skor: {max(silhouette_scores):.3f})")

    return elbow_k, best_silhouette_k, sse, silhouette_scores

# Optimal küme sayısını bul
elbow_k, silhouette_k, sse_values, sil_scores = find_optimal_clusters(X_scaled)

# Final küme sayısına karar ver (genellikle silhouette score tercih edilir)
final_k = silhouette_k
print(f"\n🎯 Final karar: {final_k} küme kullanılacak")

# 2. Final clustering
kmeans_final = KMeans(n_clusters=final_k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_scaled)

# Sonuçları ana veri setine ekle
customer_data['Cluster'] = cluster_labels

# 3. Küme merkezlerini analiz et
cluster_centers = kmeans_final.cluster_centers_
cluster_centers_df = pd.DataFrame(cluster_centers, columns=clustering_features)

print(f"\n🎯 KÜME MERKEZLERİ (Standardize Edilmiş)")
print("="*60)
print(cluster_centers_df.round(3))

# Orijinal ölçekte küme istatistikleri
print(f"\n📊 KÜME İSTATİSTİKLERİ (Orijinal Ölçek)")
print("="*60)

cluster_stats = customer_data.groupby('Cluster')[clustering_features].agg(['mean', 'count']).round(2)
print(cluster_stats)

# 4. Küme açıklamaları oluştur
def describe_clusters(customer_data, cluster_stats):
    """Her kümenin karakteristiklerini açıkla"""

    descriptions = {}

    for cluster_id in range(final_k):
        cluster_data = cluster_stats.loc[cluster_id]

        # Her küme için temel karakteristikler
        recency = cluster_data[('Recency', 'mean')]
        frequency = cluster_data[('Frequency', 'mean')]
        monetary = cluster_data[('Monetary', 'mean')]
        count = cluster_data[('Recency', 'count')]

        # Küme tipini belirle
        if recency < 50 and frequency > 5 and monetary > 1000:
            cluster_type = "🌟 VIP Müşteriler"
            desc = "Yüksek değerli, sık alışveriş yapan, yakın zamanda aktif müşteriler"
        elif recency < 100 and frequency > 2:
            cluster_type = "💚 Sadık Müşteriler"
            desc = "Düzenli alışveriş yapan, orta-yüksek değerli müşteriler"
        elif recency > 200:
            cluster_type = "😴 Kayıp Müşteriler"
            desc = "Uzun süredir alışveriş yapmayan, geri kazanılması gereken müşteriler"
        elif frequency == 1:
            cluster_type = "🆕 Yeni Müşteriler"
            desc = "Tek seferlik alışveriş yapmış, potansiyel müşteriler"
        else:
            cluster_type = "📊 Standart Müşteriler"
            desc = "Ortalama davranış sergileyen müşteriler"

        descriptions[cluster_id] = {
            'type': cluster_type,
            'description': desc,
            'count': int(count),
            'avg_recency': round(recency, 1),
            'avg_frequency': round(frequency, 1),
            'avg_monetary': round(monetary, 2)
        }

    return descriptions

cluster_descriptions = describe_clusters(customer_data, cluster_stats)

print(f"\n🎯 KÜME AÇIKLAMALARI")
print("="*60)

for cluster_id, info in cluster_descriptions.items():
    print(f"\nKüme {cluster_id}: {info['type']}")
    print(f"  📝 {info['description']}")
    print(f"  👥 Müşteri sayısı: {info['count']}")
    print(f"  📅 Ortalama recency: {info['avg_recency']} gün")
    print(f"  🔄 Ortalama frequency: {info['avg_frequency']} sipariş")
    print(f"  💰 Ortalama monetary: ${info['avg_monetary']}")

# 5. PCA ile görselleştirme
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Görselleştirme için plot oluştur (kod örneği)
plot_code = f'''
# Kümelerin görselleştirilmesi
plt.figure(figsize=(15, 10))

# 1. PCA ile küme görselleştirme
plt.subplot(2, 2, 1)
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray'][:final_k]
for i in range(final_k):
    cluster_points = X_pca[cluster_labels == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1],
                c=colors[i], label=f'Küme {{i}}', alpha=0.6)

plt.xlabel(f'PC1 ({{pca.explained_variance_ratio_[0]:.1%}} varyans)')
plt.ylabel(f'PC2 ({{pca.explained_variance_ratio_[1]:.1%}} varyans)')
plt.title('PCA ile Müşteri Kümeleri')
plt.legend()

# 2. RFM dağılımları
plt.subplot(2, 2, 2)
customer_data.boxplot(column='Monetary', by='Cluster', ax=plt.gca())
plt.title('Kümeler Arası Monetary Dağılımı')
plt.ylabel('Monetary Value')

plt.subplot(2, 2, 3)
customer_data.boxplot(column='Frequency', by='Cluster', ax=plt.gca())
plt.title('Kümeler Arası Frequency Dağılımı')
plt.ylabel('Purchase Frequency')

plt.subplot(2, 2, 4)
customer_data.boxplot(column='Recency', by='Cluster', ax=plt.gca())
plt.title('Kümeler Arası Recency Dağılımı')
plt.ylabel('Days Since Last Purchase')

plt.tight_layout()
plt.savefig('../results/cluster_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
'''

print(f"\n📊 Görselleştirme kodu hazır!")
print(f"💾 PCA açıklanan varyans: PC1={pca.explained_variance_ratio_[0]:.1%}, PC2={pca.explained_variance_ratio_[1]:.1%}")

# Sonuçları kaydet
customer_data.to_csv('../results/customer_clusters.csv', index=False)
cluster_centers_df.to_csv('../results/cluster_centers.csv', index=True)

# Küme özetini kaydet
cluster_summary = []
for cluster_id, info in cluster_descriptions.items():
    cluster_summary.append({
        'Cluster_ID': cluster_id,
        'Type': info['type'],
        'Description': info['description'],
        'Customer_Count': info['count'],
        'Avg_Recency_Days': info['avg_recency'],
        'Avg_Frequency': info['avg_frequency'],
        'Avg_Monetary': info['avg_monetary']
    })

pd.DataFrame(cluster_summary).to_csv('../results/cluster_summary.csv', index=False)

print(f"\n✅ Kümeleme analizi tamamlandı!")
print(f"📁 Sonuçlar 'results/' klasöründe kaydedildi")
print(f"🎯 Toplam {final_k} küme oluşturuldu, {len(customer_data)} müşteri kümelendi")

🎯 MÜŞTERİ KÜMELEME ANALİZİ
📊 1023 müşteri, 8 özellik ile kümeleme yapılıyor
🔍 Optimal küme sayısı aranıyor...
K=2: SSE=6103, Silhouette=0.561
K=3: SSE=4557, Silhouette=0.532
K=4: SSE=3810, Silhouette=0.364
K=5: SSE=3179, Silhouette=0.398
K=6: SSE=2796, Silhouette=0.370
K=7: SSE=2326, Silhouette=0.385
K=8: SSE=2028, Silhouette=0.429
K=9: SSE=1847, Silhouette=0.378
K=10: SSE=1691, Silhouette=0.398
K=11: SSE=1528, Silhouette=0.394
K=12: SSE=1420, Silhouette=0.396
K=13: SSE=1323, Silhouette=0.337
K=14: SSE=1226, Silhouette=0.339
K=15: SSE=1179, Silhouette=0.337

📈 Elbow method önerisi: 7 küme
📈 En iyi Silhouette score: 2 küme (skor: 0.561)

🎯 Final karar: 2 küme kullanılacak

🎯 KÜME MERKEZLERİ (Standardize Edilmiş)
   Recency  Frequency  Monetary  Total_Spent  Avg_Order_Value  \
0    0.238     -0.241    -0.131       -0.131           -0.041   
1   -1.607      1.630     0.883        0.883            0.277   

   Unique_Products  Customer_Lifespan_Days  Purchase_Frequency  
0           -0.160